# Les Arc ou compteurs atomiques de référence

---

Il existe encore beaucoup d'autres structures en Rust qui sont allouées dynamiquement, faire la description de toutes ces structures serait une perte de temps dans le cadre de ce cours, c'est votre pratique future du Rust qui vous les enseignera petit à petit.

Pour clore ce chapitre, je vais brièvement présenter une structure un peu plus atypique, que vous croiserez peut être moins souvent, l'Arc ou Atomically Reference Counter.

C'est une structure allouée sur le tas et qui est destinée à être partagée entres différents threads. Elle n'est donc a utiliser que dans le cadre des programmes à fils multiples.

```
Struct std::sync::Arc

pub struct Arc<T>
where
    T: ?Sized,
{ /* private fields */ }

A thread-safe reference-counting pointer. ‘Arc’ stands for ‘Atomically Reference Counted’.
```

## Caractéristiques des Arc

> - Arc implémente le trait clone, chaque thread peut accéder indépendamment à son clone de l'Arc, chaque clone pointe sur le même endroit mémoire, le type contenu par l'Arc.

> - Un compteur de référence interne fonctionne ainsi : pour chaque clone qui est fabriqué, il est incrémenté de 1, pour chaque clone détruit (par la fin d'un bloc d'instruction, d'un appel à Drop ou par une fonction qui ne le retourne pas), il est soustrait de 1.

> - Quand le compteur tombe a 0, l'Arc et le type contenu sont libérés de la mémoire.

> - Le compteur étant dit Atomic, il ne peut pas y avoir de problème de concurrence lors de l’incrémentation ou de la décrémentation de ce dernier.

> - Enfin, un Arc n'est pas un mutex et ne peut à lui tout seul permettre un accès mutable au type contenu. Afin de pouvoir partager des donnees mutables, on utilise l'Arc en conjonction avec un Mutex : **Arc\<Mutex\<T>>** 

## Proof of concept

- Tout comme les HashMap, Arc ne fait pas partie du prélude, il faut donc l'importer. Voici un exemple très simple qui illustre les changements du compteur de référence, mais dans un environnement mono-thread. Le multithreading sera aborde chapitre 9 :

In [12]:
use std::sync::Arc;

#[derive(Debug)]
struct Vector {
    i: u32,
    j: u32,
}
// Creation de l'Arc, le reference Counter passe a 1
let a = Arc::<Vector>::new(Vector {i: 12, j: 42});
println!("Compteur de reference juste apres la creation: {}", Arc::strong_count(&a));
{
    // Creation du clone, le reference Counter passe a 2
    let b = a.clone();
    println!("Compteur de reference apres le clonage: {}", Arc::strong_count(&b));
    // Passage par reference, le clone n'est pas detruit lors de l'appel
    dbg!(&b);
        
    println!("Compteur de reference avant la fin du sous-bloc: {}", Arc::strong_count(&b));
    // Ici le clone est detruit -> Fin du bloc d'instruction
    // Le reference Counter repasse a 1
}
// L'arc est envoye a la fonction dbg!, il est ainsi detruit apres le retour
// de la fonction dbg! puisque son reference Counter passe a 0.
println!("Compteur de reference a la fin de la fonction principale: {}", Arc::strong_count(&a));
dbg!(a);

Compteur de reference juste apres la creation: 1
Compteur de reference apres le clonage: 2
Compteur de reference avant la fin du sous-bloc: 2
Compteur de reference a la fin de la fonction principale: 1


[src/lib.rs:20] &b = Vector {
    i: 12,
    j: 42,
}
[src/lib.rs:29] a = Vector {
    i: 12,
    j: 42,
}


> **N'hésitez pas à lire la documentation de Rust pour des illustrations dans le cadre du multi-threading.**